In [ ]:
import csv
import numpy as np
import networkx as nx
import math
from scipy.stats import mode
import function
import pickle

# man_dis,lat_different,long_different,direction

In [ ]:
man_dis_dic = {}
direction_dic = {}
lat_diff_dic = {}
long_diff_dic = {}
user_show_count_dic = {}

i = 0
csv_reader = csv.reader(open('./data/train_hard3.csv', encoding='utf-8'))
for row in csv_reader:
    i+=1
    if i == 1:
        print(row[9],row[18],row[19],row[20],row[21])
        continue
        
    user = row[1]
    start = row[5]
    end = row[6]
        
    man_dis = row[9]
    direction = row[18]
    lat_diff = row[19]
    long_diff = row[20]
    user_show_count = row[21]
    
    if start not in man_dis_dic:
        man_dis_dic[start] = {}
        man_dis_dic[start][end] = man_dis
    else:
        if end not in man_dis_dic[start]:
            man_dis_dic[start][end] = man_dis
    
    if start not in direction_dic:
        direction_dic[start] = {}
        direction_dic[start][end] = direction
    else:
        if end not in direction_dic[start]:
            direction_dic[start][end] = direction
            
    if start not in lat_diff_dic:
        lat_diff_dic[start] = {}
        lat_diff_dic[start][end] = lat_diff
    else:
        if end not in lat_diff_dic[start]:
            lat_diff_dic[start][end] = lat_diff
            
    if start not in long_diff_dic:
        long_diff_dic[start] = {}
        long_diff_dic[start][end] = long_diff
    else:
        if end not in long_diff_dic[start]:
            long_diff_dic[start][end] = long_diff
            
    if user not in user_show_count_dic:
        user_show_count_dic[user] = user_show_count
    
    
    

In [ ]:
  
with open('./data/temp/Incomplete_caching/man_dis_dic','wb+') as f:
    pickle.dump(man_dis_dic, f, pickle.HIGHEST_PROTOCOL)
    
with open('./data/temp/Incomplete_caching/direction_dic','wb+') as f:
    pickle.dump(direction_dic, f, pickle.HIGHEST_PROTOCOL)


with open('./data/temp/Incomplete_caching/lat_diff_dic','wb+') as f:
    pickle.dump(lat_diff_dic, f, pickle.HIGHEST_PROTOCOL)
    
with open('./data/temp/Incomplete_caching/long_diff_dic','wb+') as f:
    pickle.dump(long_diff_dic, f, pickle.HIGHEST_PROTOCOL)
    
with open('./data/temp/Incomplete_caching/user_show_count_dic','wb+') as f:
    pickle.dump(user_show_count_dic, f, pickle.HIGHEST_PROTOCOL)
    



## 生成用户的使用序列

In [ ]:
import pandas as pd

In [ ]:
PATH= './IJCAI/'
df_train = pd.read_csv('./data/train_hard.csv')
df_test = pd.read_csv('./data/test_hard.csv')

In [ ]:
length = len(df_train)
df_merge = pd.concat([df_train, df_test]).reset_index()
df_merge.drop(['index'],axis=1,inplace=True)

In [ ]:
df_merge = df_merge.iloc[df_merge.starttime.sort_values().index]

In [ ]:
user_used_time = {}

for user in df_merge.userid.value_counts().index:
    user_used_time[user] = df_merge[df_merge.userid == user]


In [ ]:
with open('./data/temp/Incomplete_caching/user_used_time','wb+') as f:
    pickle.dump(user_used_time, f, pickle.HIGHEST_PROTOCOL)

# part1

## user_to_time,user_to_geofly_time


In [ ]:
def get_user_end_hour_dic(path):
    
    user_hour_dic = {}

    i = 0
    csv_reader = csv.reader(open(path, encoding='utf-8'))
    for row in csv_reader:
        i+=1
        if i == 1:
            continue
        
        user = row[1]
        end = row[6]
        hour = row[7]
        
        if user not in user_hour_dic:
            user_hour_dic[user] = {}
            user_hour_dic[user][end] = set()
            user_hour_dic[user][end].add(int(hour))
        else:
            if end not in user_hour_dic[user]:
                user_hour_dic[user][end] = set()
                user_hour_dic[user][end].add(int(hour))
            else:
                if int(hour) not in user_hour_dic[user][end]:
                    user_hour_dic[user][end].add(int(hour))
    return user_hour_dic

In [ ]:
user_to_time = get_user_end_hour_dic('./data/train_hard.csv')

user_to_geofly_time = get_user_end_hour_dic('./data/train_end_geofly.csv')

In [ ]:
with open('./data/temp/Incomplete_caching/user_to_time','wb+') as f:
    pickle.dump(user_to_time, f, pickle.HIGHEST_PROTOCOL)
    
with open('./data/temp/Incomplete_caching/user_to_geofly_time','wb+') as f:
    pickle.dump(user_to_geofly_time, f, pickle.HIGHEST_PROTOCOL)


## csv append

In [ ]:
def user_to_min_time_diff(user_hour_dic_path,user_hour_dic_path_geofly,append_path):
    user_hour_dic = get_user_end_hour_dic(user_hour_dic_path)
    user_hour_dic_geofly = get_user_end_hour_dic(user_hour_dic_path_geofly)
    
    with open(output_path, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        
        i = 0
        csv_reader = csv.reader(open(append_path, encoding='utf-8'))
        for row in csv_reader:
            i+=1
            if i == 1:
                row.append('user_to_end_min_time_diff')
                row.append('user_to_end_geofly_min_time_diff')
                row.append('user_to_start_min_time_diff')
                row.append('user_to_start_geofly_min_time_diff')
                writer.writerow(row)
                continue
            user = row[1]
            start = row[5]
            end = row[6]
            hour = row[7]

            if user in user_hour_dic and end in user_hour_dic[user]:# 用户去此终点的时间与历史时间的最小时间差
                mindiff = 100
                for x in user_hour_dic[user][end]:
                    if abs(x - int(hour)) < mindiff:
                        mindiff = abs(x - int(hour))
                row.append(mindiff)
            else:
                row.append('')
                
            # 用户去此终点（扩大9倍）的时间与历史时间的最小时间差
            if user in user_hour_dic_geofly and end in user_hour_dic_geofly[user]:
                mindiff = 100
                for x in user_hour_dic_geofly[user][end]:
                    if abs(x - int(hour)) < mindiff:
                        mindiff = abs(x - int(hour))
                row.append(mindiff)
            else:
                row.append('')
                
                
            if user in user_hour_dic and start in user_hour_dic[user]:# 用户去此起点的时间与历史时间的最小时间差
                mindiff = 100
                for x in user_hour_dic[user][start]:
                    if abs(x - int(hour)) < mindiff:
                        mindiff = abs(x - int(hour))
                row.append(mindiff)
            else:
                row.append('')
                
            # 用户去此起点（扩大9倍）的时间与历史时间的最小时间差
            if user in user_hour_dic_geofly and start in user_hour_dic_geofly[user]:
                mindiff = 100
                for x in user_hour_dic_geofly[user][start]:
                    if abs(x - int(hour)) < mindiff:
                        mindiff = abs(x - int(hour))
                row.append(mindiff)
            else:
                row.append('')
    
        writer.writerow(row)
    

## user_from_time,user_from_geofly_time


In [ ]:
def get_user_start_hour_dic(path):
    
    user_hour_dic = {}

    i = 0
    csv_reader = csv.reader(open(path, encoding='utf-8'))
    for row in csv_reader:
        i+=1
        if i == 1:
            continue
        
        user = row[1]
        start = row[5]
        hour = row[7]
        
        if user not in user_hour_dic:
            user_hour_dic[user] = {}
            user_hour_dic[user][start] = set()
            user_hour_dic[user][start].add(int(hour))
        else:
            if start not in user_hour_dic[user]:
                user_hour_dic[user][start] = set()
                user_hour_dic[user][start].add(int(hour))
            else:
                if int(hour) not in user_hour_dic[user][start]:
                    user_hour_dic[user][start].add(int(hour))
    return user_hour_dic

In [ ]:
user_from_time = get_user_start_hour_dic('./data/train_hard.csv')

user_from_geofly_time = get_user_start_hour_dic('./data/train_start_geofly.csv')

In [ ]:
with open('./data/temp/Incomplete_caching/user_from_time','wb+') as f:
    pickle.dump(user_from_time, f, pickle.HIGHEST_PROTOCOL)
    
with open('./data/temp/Incomplete_caching/user_from_geofly_time','wb+') as f:
    pickle.dump(user_from_geofly_time, f, pickle.HIGHEST_PROTOCOL)



## csv append

In [ ]:
def user_to_min_time_diff(user_hour_dic_path,user_hour_dic_path_geofly,append_path,output_path):
    user_hour_dic = get_user_start_hour_dic(user_hour_dic_path)
    user_hour_dic_geofly = get_user_start_hour_dic(user_hour_dic_path_geofly)
    
    with open(output_path, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        
        i = 0
        csv_reader = csv.reader(open(append_path, encoding='utf-8'))
        for row in csv_reader:
            i+=1
            if i == 1:
                row.append('user_from_end_min_time_diff')
                row.append('user_from_end_geofly_min_time_diff')
                row.append('user_from_start_min_time_diff')
                row.append('user_from_start_geofly_min_time_diff')
                writer.writerow(row)
                continue
            user = row[1]
            start = row[5]
            end = row[6]
            hour = row[7]

            if user in user_hour_dic and end in user_hour_dic[user]:# 用户从此终点出发的时间与历史时间的最小时间差
                mindiff = 100
                for x in user_hour_dic[user][end]:
                    if abs(x - int(hour)) < mindiff:
                        mindiff = abs(x - int(hour))
                row.append(mindiff)
            else:
                row.append('')
                
            # 用户从此终点出发（扩大9倍）的时间与历史时间的最小时间差
            if user in user_hour_dic_geofly and end in user_hour_dic_geofly[user]:
                mindiff = 100
                for x in user_hour_dic_geofly[user][end]:
                    if abs(x - int(hour)) < mindiff:
                        mindiff = abs(x - int(hour))
                row.append(mindiff)
            else:
                row.append('')
                
                
            if user in user_hour_dic and start in user_hour_dic[user]:# 用户从此起点的时间与历史时间的最小时间差
                mindiff = 100
                for x in user_hour_dic[user][start]:
                    if abs(x - int(hour)) < mindiff:
                        mindiff = abs(x - int(hour))
                row.append(mindiff)
            else:
                row.append('')
                
            # 用户从此起点（扩大9倍）的时间与历史时间的最小时间差
            if user in user_hour_dic_geofly and start in user_hour_dic_geofly[user]:
                mindiff = 100
                for x in user_hour_dic_geofly[user][start]:
                    if abs(x - int(hour)) < mindiff:
                        mindiff = abs(x - int(hour))
                row.append(mindiff)
            else:
                row.append('')
    
        writer.writerow(row)
    

# part2

In [ ]:
def get_user_start_to_end_hour_dic(path):
    user_hour_dic = {}

    i = 0
    csv_reader = csv.reader(open(path, encoding='utf-8'))
    for row in csv_reader:
        i+=1
        if i == 1:
            continue
        
        user = row[1]
        start = row[5]
        end = row[6]
        hour = row[7]
            
        
        if user not in user_hour_dic:
            user_hour_dic[user] = {}
            user_hour_dic[user][start] = {}
            user_hour_dic[user][start][end] = set()
            user_hour_dic[user][start][end].add(int(hour))
        else:
            if start not in user_hour_dic[user]:
                user_hour_dic[user][start] = {}
                user_hour_dic[user][start][end] = set()
                user_hour_dic[user][start][end].add(int(hour))
            else:

                if end not in user_hour_dic[user][start]:
                    user_hour_dic[user][start][end] = set()
                    user_hour_dic[user][start][end].add(int(hour))
                else:
                    user_hour_dic[user][start][end].add(int(hour))
    return user_hour_dic

In [ ]:
user_start_to_end_time = get_user_start_to_end_hour_dic('./data/train_hard.csv')

In [ ]:
user_start_to_end_geofly_time = get_user_start_to_end_hour_dic('./data/train_end_geofly.csv')

In [ ]:
user_start_geofly_to_end_time = get_user_start_to_end_hour_dic('./data/train_start_geofly.csv')

In [ ]:
with open('./data/temp/Incomplete_caching/user_start_to_end_time','wb+') as f:
    pickle.dump(user_start_to_end_time, f, pickle.HIGHEST_PROTOCOL)
    
with open('./data/temp/Incomplete_caching/user_start_to_end_geofly_time','wb+') as f:
    pickle.dump(user_start_to_end_geofly_time, f, pickle.HIGHEST_PROTOCOL)


with open('./data/temp/Incomplete_caching/user_start_geofly_to_end_time','wb+') as f:
    pickle.dump(user_start_geofly_to_end_time, f, pickle.HIGHEST_PROTOCOL)
  

# 方向

In [ ]:
def get_direction(path):
    with open("./data/train_hard1.csv", 'w', newline='') as csv_file:
        
        #test_hard1
        writer = csv.writer(csv_file)
        i = 0
        csv_reader = csv.reader(open(path, encoding='utf-8'))
        for row in csv_reader:
            i+=1
            if i == 1:
                row.append('direction')
                writer.writerow(row)
                continue
            start = row[5]
            end = row[6]

            direction = function.getDegree(start,end)
            
            if direction < 22.5 or 337.5 <= direction:
                direction = 0
            elif 22.5 <= direction and direction < 67.5:
                direction = 1
            elif 67.5 <= direction and direction < 112.5:
                direction = 2
            elif 112.5 <= direction and direction < 157.5:
                direction = 3
            elif 157.5 <= direction and direction < 202.5:
                direction = 4
            elif 202.5 <= direction and direction < 247.5:
                direction = 5
            elif 247.5 <= direction and direction < 292.5:
                direction = 6
            elif 292.5 <= direction and direction < 337.5:
                direction = 7
                
            row.append(direction)
            writer.writerow(row)

In [ ]:
get_direction('./data/train_hard.csv')

# 经度差和纬度差

In [ ]:
function.decode_exactly('wxsws4')

In [ ]:
def creat_long_lat_diff(path):
    with open("./data/train_hard2.csv", 'w', newline='') as csv_file:
        
        #test_hard1
        writer = csv.writer(csv_file)
        i = 0
        csv_reader = csv.reader(open(path, encoding='utf-8'))
        for row in csv_reader:
            i+=1
            if i == 1:
                row.append('lat_diff')
                row.append('long_diff')
                writer.writerow(row)
                continue
            start = row[5]
            end = row[6]
            
            lat1 = function.decode_exactly(start)[0]
            long1 = function.decode_exactly(start)[1]
            
            lat2 = function.decode_exactly(end)[0]
            long2 = function.decode_exactly(end)[1]

            
            row.append(abs(lat1 - lat2))
            row.append(abs(long1 - long2))
            writer.writerow(row)

In [ ]:
creat_long_lat_diff('./data/train_hard1.csv')